In [1]:
!pip install causalml 

In [21]:
from flask import Flask, request, render_template_string
import pandas as pd
import numpy as np
from econml.dml import LinearDML
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
import matplotlib.pyplot as plt
import io
import base64
import threading

app = Flask(__name__)

In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
import io
import base64

def perform_psm(df):
    try:
        
        X = df[['age', 'gender']]
        T = df['treatment']
        Y = df['outcome']
        df['gender'] = df['gender'].map({'male': 1, 'female': 0})

      
        model = LogisticRegression()
        model.fit(X, T)
        df['propensity_score'] = model.predict_proba(X)[:, 1]

       
        treated = df[df['treatment'] == 1].copy()
        control = df[df['treatment'] == 0].copy()

        nn = NearestNeighbors(n_neighbors=1)
        nn.fit(control[['propensity_score']])
        distances, indices = nn.kneighbors(treated[['propensity_score']])
        matched_control = control.iloc[indices.flatten()].copy()
        matched = pd.concat([treated, matched_control])

        
        ate = treated['outcome'].mean() - matched_control['outcome'].mean()

    
        fig, ax = plt.subplots(figsize=(8, 4))
        sns.kdeplot(df['age'], label='Before Matching', ax=ax)
        sns.kdeplot(matched['age'], label='After Matching', ax=ax)
        ax.set_title("Age Distribution Before vs After Matching")
        ax.legend()

       
        buf = io.BytesIO()
        plt.tight_layout()
        plt.savefig(buf, format='png')
        buf.seek(0)
        image_base64 = base64.b64encode(buf.read()).decode('utf-8')
        buf.close()
        plt.close(fig)

        return round(ate, 4), image_base64
    except Exception as e:
        return f"Error during PSM analysis: {e}", None


In [23]:
upload_form = """
<!doctype html>
<html lang="en">
<head>
    <title>PSM ATE Calculator</title>
    <style>
        :root {
            --primary: #4361ee;
            --secondary: #3f37c9;
            --success: #4cc9f0;
            --light: #f8f9fa;
            --dark: #212529;
            --danger: #f72585;
            --border-radius: 8px;
            --box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
            --transition: all 0.3s ease;
        }
        
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }
        
        body {
            background-color: #f5f7fa;
            color: var(--dark);
            line-height: 1.6;
            padding: 20px;
            max-width: 800px;
            margin: 0 auto;
        }
        
        .container {
            background-color: white;
            border-radius: var(--border-radius);
            box-shadow: var(--box-shadow);
            padding: 30px;
            margin-top: 30px;
        }
        
        h2 {
            color: var(--primary);
            margin-bottom: 20px;
            text-align: center;
            font-weight: 600;
        }
        
        .upload-form {
            display: flex;
            flex-direction: column;
            gap: 20px;
            margin-bottom: 30px;
        }
        
        .file-input-wrapper {
            position: relative;
            overflow: hidden;
            display: inline-block;
            width: 100%;
        }
        
        .file-input-button {
            border: 2px dashed #ccc;
            border-radius: var(--border-radius);
            padding: 40px 20px;
            text-align: center;
            cursor: pointer;
            transition: var(--transition);
            background-color: var(--light);
        }
        
        .file-input-button:hover {
            border-color: var(--primary);
            background-color: rgba(67, 97, 238, 0.05);
        }
        
        .file-input-wrapper input[type="file"] {
            position: absolute;
            left: 0;
            top: 0;
            opacity: 0;
            width: 100%;
            height: 100%;
            cursor: pointer;
        }
        
        .submit-btn {
            background-color: var(--primary);
            color: white;
            border: none;
            padding: 12px 20px;
            border-radius: var(--border-radius);
            cursor: pointer;
            font-size: 16px;
            font-weight: 500;
            transition: var(--transition);
            width: 100%;
        }
        
        .submit-btn:hover {
            background-color: var(--secondary);
            transform: translateY(-2px);
        }
        
        .results {
            margin-top: 30px;
            padding: 20px;
            background-color: var(--light);
            border-radius: var(--border-radius);
            text-align: center;
            animation: fadeIn 0.5s ease;
        }
        
        .results h3 {
            color: var(--secondary);
            margin-bottom: 15px;
            font-size: 24px;
        }
        
        .results img {
            max-width: 100%;
            border-radius: var(--border-radius);
            box-shadow: var(--box-shadow);
            margin-top: 20px;
            border: 1px solid #eee;
        }
        
        .error {
            color: var(--danger);
            background-color: rgba(247, 37, 133, 0.1);
            padding: 15px;
            border-radius: var(--border-radius);
            margin-top: 20px;
            text-align: center;
            border-left: 4px solid var(--danger);
        }
        
        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(10px); }
            to { opacity: 1; transform: translateY(0); }
        }
        
        .file-name {
            margin-top: 10px;
            font-size: 14px;
            color: #666;
        }
    </style>
</head>
<body>
    <div class="container">
        <h2>Upload CSV File for Propensity Score Matching</h2>
        
        <form method="post" enctype="multipart/form-data" class="upload-form">
            <div class="file-input-wrapper">
                <label class="file-input-button">
                    <span>Choose CSV file or drag and drop here</span>
                    <input type="file" name="csv_file" accept=".csv" required>
                </label>
                <div class="file-name" id="file-name">No file selected</div>
            </div>
            <button type="submit" class="submit-btn">Calculate ATE</button>
        </form>
        
        {% if ate %}
            <div class="results">
                <h3>Average Treatment Effect: <span style="color: var(--primary);">{{ ate }}</span></h3>
                <img src="data:image/png;base64,{{ plot_data }}" alt="Matching Results Visualization">
            </div>
        {% elif error %}
            <div class="error">{{ error }}</div>
        {% endif %}
    </div>

    <script>
        // Simple JavaScript to show selected filename (optional enhancement)
        document.querySelector('input[type="file"]').addEventListener('change', function(e) {
            const fileName = e.target.files[0] ? e.target.files[0].name : 'No file selected';
            document.getElementById('file-name').textContent = fileName;
        });
    </script>
</body>
</html>
"""


In [24]:
@app.route('/', methods=['GET', 'POST'])
def upload_and_analyze():
    ate = None
    plot_data = None
    error = None

    if request.method == 'POST':
        file = request.files.get('csv_file')
        if file and file.filename.endswith('.csv'):
            try:
                df = pd.read_csv(file)

               
                ate_result, plot_data = perform_psm(df)

                if isinstance(ate_result, str):
                    error = f"PSM Error: {ate_result}"
                else:
                    ate = f"{ate_result:.4f}"

            except Exception as e:
                error = f"Processing error: {str(e)}"
        else:
            error = "Please upload a valid CSV file."

    return render_template_string(upload_form, ate=ate, plot_data=plot_data, error=error)


In [25]:
def run_flask():
    app.run(port=5000, debug=True,use_reloader=False)
threading.Thread(target=run_flask).start()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
